#### V1

In [3]:
import cv2

# Yüz tanıma modelini yükle
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Video dosyasını aç
video_capture = cv2.VideoCapture('C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/short_faces.mp4')

while True:
    # Video'dan bir kare al
    ret, frame = video_capture.read()

    # Kareyi gri tonlamaya dönüştür
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Yüzleri tespit et
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Her yüz için bir dikdörtgen çiz
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Sonuçları göster
    cv2.imshow('Video', frame)

    # 'q' tuşuna basıldığında çıkış yap
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Temizlik yap
video_capture.release()
cv2.destroyAllWindows()


#### V2

In [6]:
import numpy as np
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords, img


def detect(img, faceCascade):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0)}
    coords, img = draw_boundary(img, faceCascade, 1.1, 10, color['blue'],"Face")
    return img
    
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
video_capture = cv2.VideoCapture('C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/short_faces.mp4')

while True:
    _, img = video_capture.read()
    img = detect(img, faceCascade)
    
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
video_capture.release()
cv2.destroyAllWindows()

#### V3

In [12]:
import cv2
import numpy as np

# YOLOv4 modelini yükle
net = cv2.dnn.readNet("C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/yolov4.weights", 
                      "C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Sınıf etiketlerini yükle
with open("C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Video yakalama
video_capture = cv2.VideoCapture('C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/short_faces.mp4')

while True:
    ret, img = video_capture.read()
    if not ret:
        break

    height, width, channels = img.shape

    # YOLO için görüntü hazırla
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Tespit edilen nesneleri işle
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":  # Yalnızca insanları tespit et
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (255, 0, 0)  # Mavi renk
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
